***Challenge 1***

Here the goal is to train on 100 samples. In this preliminary testbed the evaluation will be done on a 2000 sample validation set. Note in the end the final evaluation will be done on the full CIFAR-10 test set as well as potentially a separate dataset. The validation samples here should not be used for training in any way, the final evaluation will provide only random samples of 100 from a datasource that is not the CIFAR-10 training data. 

Feel free to modify this testbed to your liking, including the normalization transformations etc. Note however the final evaluation testbed will have a rigid set of components where you will need to place your answer. The only constraint is the data. Refer to the full project instructions for more information.


Setup training functions. Again you are free to fully modify this testbed in your prototyping within the constraints of the data used. You can use tools outside of pytorch for training models if desired as well although the torchvision dataloaders will still be useful for interacting with the cifar-10 dataset. 

In [ ]:
def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    if display:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [ ]:
import torch.nn as nn 
import torch.nn.functional as F

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.ModuleList()
        
        self.layers+=[nn.Conv2d(3, 16,  kernel_size=3) , 
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(16, 16,  kernel_size=3, stride=2), 
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(16, 32,  kernel_size=3), 
                      nn.ReLU(inplace=True)]
        self.layers+=[nn.Conv2d(32, 32,  kernel_size=3, stride=2), 
                      nn.ReLU(inplace=True)]
        self.fc = nn.Linear(32*5*5, 10)
    def forward(self, x):
        for i in range(len(self.layers)):
          x = self.layers[i](x)
        x = x.view(-1, 32*5*5)
        x = self.fc(x)
        return x

The below tries  2 random problem instances. In your development you may choose to prototype with 1 problem instances but keep in mind for small sample problems the variance is high so continously evaluating on several subsets will be important.

In [ ]:
from numpy.random import RandomState
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Subset

  
from torchvision import datasets, transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([transforms.ToTensor(), normalize]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
    
#We need two copies of this due to weird dataset api 
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)
    
# Extract a subset of 100 (class balanced) samples per class

accs = []

for seed in [1,2]:
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 5000))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:10]] for classe in range(0, 10)])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[10:210]] for classe in range(0, 10)])


  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))
  
  train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)
  

  model = Net()
  model.to(device)
  optimizer = torch.optim.SGD(model.parameters(), 
                              lr=0.01, momentum=0.9,
                              weight_decay=0.0005)
  for epoch in range(100):
    train(model, device, train_loader, optimizer, epoch, display=epoch%5==0)
    
  accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over 2 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))


Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 100 Num Samples For Val 2000
Train Epoch: 0 [0/100 (0%)]	Loss: 2.302286
Train Epoch: 5 [0/100 (0%)]	Loss: 2.301360
Train Epoch: 10 [0/100 (0%)]	Loss: 2.299233
Train Epoch: 15 [0/100 (0%)]	Loss: 2.296285
Train Epoch: 20 [0/100 (0%)]	Loss: 2.292279
Train Epoch: 25 [0/100 (0%)]	Loss: 2.286338
Train Epoch: 30 [0/100 (0%)]	Loss: 2.276244
Train Epoch: 35 [0/100 (0%)]	Loss: 2.255733
Train Epoch: 40 [0/100 (0%)]	Loss: 2.211310
Train Epoch: 45 [0/100 (0%)]	Loss: 2.154057
Train Epoch: 50 [0/100 (0%)]	Loss: 2.054544
Train Epoch: 55 [0/100 (0%)]	Loss: 1.912884
Train Epoch: 60 [0/100 (0%)]	Loss: 1.745162
Train Epoch: 65 [0/100 (0%)]	Loss: 1.548557
Train Epoch: 70 [0/100 (0%)]	Loss: 1.361159
Train Epoch: 75 [0/100 (0%)]	Loss: 1.323507
Train Epoch: 80 [0/100 (0%)]	Loss: 1.412675
Train Epoch: 85 [0/100 (0%)]	Loss: 1.078649
Train Epoch: 90 [0/100 (0%)]	Loss: 0.842692
Train Epoch: 95 [0/100 (0%)]	Loss: 0

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 7.5566, Accuracy: 384/2000 (19.20%)

Num Samples For Training 100 Num Samples For Val 2000
Train Epoch: 0 [0/100 (0%)]	Loss: 2.302706
Train Epoch: 5 [0/100 (0%)]	Loss: 2.301484
Train Epoch: 10 [0/100 (0%)]	Loss: 2.298963
Train Epoch: 15 [0/100 (0%)]	Loss: 2.295718
Train Epoch: 20 [0/100 (0%)]	Loss: 2.291164
Train Epoch: 25 [0/100 (0%)]	Loss: 2.283609
Train Epoch: 30 [0/100 (0%)]	Loss: 2.268948
Train Epoch: 35 [0/100 (0%)]	Loss: 2.236839
Train Epoch: 40 [0/100 (0%)]	Loss: 2.181045
Train Epoch: 45 [0/100 (0%)]	Loss: 2.110200
Train Epoch: 50 [0/100 (0%)]	Loss: 2.009273
Train Epoch: 55 [0/100 (0%)]	Loss: 1.900102
Train Epoch: 60 [0/100 (0%)]	Loss: 1.762625
Train Epoch: 65 [0/100 (0%)]	Loss: 1.578496
Train Epoch: 70 [0/100 (0%)]	Loss: 1.323992
Train Epoch: 75 [0/100 (0%)]	Loss: 2.475161
Train Epoch: 80 [0/100 (0%)]	Loss: 2.053359
Train Epoch: 85 [0/100 (0%)]	Loss: 2.142391
Train Epoch: 90 [0/100 (0%)]	Loss: 2.037991
Train Epoch: 95 [0/100 (0%)]	Loss: 1.769239

Test 

***Challenge 2***

You may use the same testbed but without the constraints on external datasets or models trained on exeternal datasets. You may not however use any of the CIFAR-10 training set. 

In [ ]:
import torchvision.models as models



In [ ]:
from numpy.random import RandomState
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Subset

  
from torchvision import datasets, transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])

# We resize images to allow using imagenet pre-trained models, is there a better way?
resize = transforms.Resize(224) 

transform_val = transforms.Compose([resize, transforms.ToTensor(), normalize]) #careful to keep this one same
transform_train = transforms.Compose([resize, transforms.ToTensor(), normalize]) 

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device) # you will really need gpu's for this part
    
##### Cifar Data
cifar_data = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
    
#We need two copies of this due to weird dataset api 
cifar_data_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)
    
# Extract a subset of 100 (class balanced) samples per class

accs = []

for seed in [1,2]:
  prng = RandomState(seed)
  random_permute = prng.permutation(np.arange(0, 5000))
  indx_train = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[0:10]] for classe in range(0, 10)])
  indx_val = np.concatenate([np.where(np.array(cifar_data.targets) == classe)[0][random_permute[10:210]] for classe in range(0, 10)])

  train_data = Subset(cifar_data, indx_train)
  val_data = Subset(cifar_data_val, indx_val)

  print('Num Samples For Training %d Num Samples For Val %d'%(train_data.indices.shape[0],val_data.indices.shape[0]))
  
  train_loader = torch.utils.data.DataLoader(train_data,
                                             batch_size=128, 
                                             shuffle=True)

  val_loader = torch.utils.data.DataLoader(val_data,
                                           batch_size=128, 
                                           shuffle=False)
  

  model = models.alexnet(pretrained=True)
  model.classifier = nn.Linear(256 * 6 * 6, 10)
  optimizer = torch.optim.SGD(model.classifier.parameters(), 
                              lr=0.01, momentum=0.9,
                              weight_decay=0.0005)
  model.to(device)
  for epoch in range(10):
    train(model, device, train_loader, optimizer, epoch, display=True)
    
  accs.append(test(model, device, val_loader))

accs = np.array(accs)
print('Acc over 2 instances: %.2f +- %.2f'%(accs.mean(),accs.std()))

cuda
Files already downloaded and verified
Files already downloaded and verified
Num Samples For Training 100 Num Samples For Val 2000
Train Epoch: 0 [0/100 (0%)]	Loss: 2.508620
Train Epoch: 1 [0/100 (0%)]	Loss: 1.984713
Train Epoch: 2 [0/100 (0%)]	Loss: 0.988828
Train Epoch: 3 [0/100 (0%)]	Loss: 0.455914
Train Epoch: 4 [0/100 (0%)]	Loss: 0.129157
Train Epoch: 5 [0/100 (0%)]	Loss: 0.085641
Train Epoch: 6 [0/100 (0%)]	Loss: 0.050821
Train Epoch: 7 [0/100 (0%)]	Loss: 0.023876
Train Epoch: 8 [0/100 (0%)]	Loss: 0.010803
Train Epoch: 9 [0/100 (0%)]	Loss: 0.005150


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 1.7421, Accuracy: 982/2000 (49.10%)

Num Samples For Training 100 Num Samples For Val 2000
Train Epoch: 0 [0/100 (0%)]	Loss: 2.355653
Train Epoch: 1 [0/100 (0%)]	Loss: 1.850445
Train Epoch: 2 [0/100 (0%)]	Loss: 0.962296
Train Epoch: 3 [0/100 (0%)]	Loss: 0.504608
Train Epoch: 4 [0/100 (0%)]	Loss: 0.173336
Train Epoch: 5 [0/100 (0%)]	Loss: 0.099966
Train Epoch: 6 [0/100 (0%)]	Loss: 0.055117
Train Epoch: 7 [0/100 (0%)]	Loss: 0.025471
Train Epoch: 8 [0/100 (0%)]	Loss: 0.012577
Train Epoch: 9 [0/100 (0%)]	Loss: 0.007205

Test set: Average loss: 1.5719, Accuracy: 1039/2000 (51.95%)

Acc over 2 instances: 50.53 +- 1.43
